In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from tqdm import tqdm


In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
device

'cuda'

# Data cleaning

## process: use relevant text for sentiment analysis primarily. if no relevant text, then use summary. if no key words in summary, then use title. If no key words at all, then...??(but i see some about dow jones...)

In [ ]:
df = pd.read_csv('drive/My Drive/thesis/news-aapl-include-text.csv')

In [ ]:
df.shape

(8263, 16)

In [ ]:
df

,Unnamed: 0.1,Unnamed: 0,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment,relevant_text
0,0,0,"us stocks fall, oil tops $105 as ukraine crisi...",https://www.aljazeera.com/economy/2022/3/1/us-...,20220301T080000,['Rita Nazareth'],a surge in oil sent shivers through risky asse...,https://www.aljazeera.com/wp-content/uploads/2...,Al Jareeza,NaN,www.aljazeera.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",-0.277460,Somewhat-Bearish,"[{'ticker': 'AAPL', 'relevance_score': '0.0627...","['In the final moments of trading , the S & P ..."
1,1,1,apple halts product sales in russia after ukra...,https://www.scmp.com/tech/big-tech/article/316...,20220302T020013,['Bloomberg'],apple inc halted sales of the iphone and its o...,https://img.i-scmp.com/cdn-cgi/image/fit=conta...,South China Morning Post,Companies,www.scmp.com,"[{'topic': 'Technology', 'relevance_score': '0...",-0.495467,Bearish,"[{'ticker': 'HPQ', 'relevance_score': '0.07817...",['Apple halts product sales in Russia after Uk...
2,2,2,russia says its economy is taking 'serious blo...,https://www.cnn.com/2022/03/02/business/russia...,20220302T080000,"['Mark Thompson and Vasco Cotovio', 'CNN Busin...",london/moscow ( cnn business ) the russian...,https://cdn.cnn.com/cnnnext/dam/assets/2203011...,CNN,Markets,www.cnn.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",-0.469431,Bearish,"[{'ticker': 'EADSF', 'relevance_score': '0.078...","['Apple , the world ’ s most valuable company ..."
3,3,3,"rich russians turn to luxury jewellery, watche...",https://www.aljazeera.com/economy/2022/3/2/ric...,20220302T080000,"['Angelina Rascouet and Andy Hoffman', 'Flavia...",with sanctions on russia sending the ruble plu...,https://www.aljazeera.com/wp-content/uploads/2...,Al Jareeza,NaN,www.aljazeera.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",-0.118323,Neutral,"[{'ticker': 'AAPL', 'relevance_score': '0.0514...",['Even as consumer brands from Apple Inc. to N...
4,4,4,"market rallies, but still must do this; 5 stoc...",https://www.investors.com/market-trend/stock-m...,20220303T080000,"[""Investor's Business Daily"", 'ED CARSON']",dow jones futures: stock market rallies on 'ni...,https://www.investors.com/wp-content/uploads/2...,Investors Business Daily,NaN,www.investors.com,"[{'topic': 'Life Sciences', 'relevance_score':...",-0.092622,Neutral,"[{'ticker': 'AAPL', 'relevance_score': '0.1497...",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8258,8258,8840,"even in an advertising slowdown, these 3 stock...",https://www.fool.com/investing/2022/12/31/even...,20221231T120000,['Adam Levy'],"ad sales growth will slow in 2023, but these s...",https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,NaN,www.fool.com,"[{'topic': 'Earnings', 'relevance_score': '0.9...",0.151938,Somewhat-Bullish,"[{'ticker': 'GOOG', 'relevance_score': '0.2433...",['able to track users across apps and the web ...
8259,8259,8841,3 unstoppable growth stocks to buy after a sto...,https://www.fool.com/investing/2022/12/31/3-un...,20221231T140000,['Dani Cook'],these companies have grown by triple-digit per...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,NaN,www.fool.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.152163,Somewhat-Bullish,"[{'ticker': 'AMD', 'relevance_score': '0.22324...","[""Apple 's dependence on China for device prod..."
8260,8260,8842,where will unity software stock be in 3 years?,https://www.fool.com/investing/2022/12/31/wher...,20221231T154500,['Leo Sun'],the game-engine developer still has a lot to p...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,NaN,www.fool.com,"[{'topic': 'IPO', 'relevance_score': '0.769861...",0.202065,Somewhat-Bullish,"[{'ticker': 'U', 'relevance_score': '0.095517'...",['Unity Ads also stopped working properly afte...
8261,8261,8843,bulls in a bear market: these 10 stock

In [ ]:
# (1) select those have relevant texts 

df_apple_relevant_text = df[df['relevant_text'] != '[]'].copy()

# dataset 1 for sentiment analysis
df_apple_relevant_text.shape

(6046, 16)

In [ ]:
# (2) select those don't have relevant texts

df_no_relevant_text = df[df['relevant_text'] == '[]'].copy()

In [ ]:
df_no_relevant_text.shape

(2217, 16)

In [ ]:
# (3) select those don't have relevant texts but have apple or aapl in title or summary

df_apple_in_ts = df_no_relevant_text[df_no_relevant_text['title'].str.contains('apple', case=False) | df_no_relevant_text['title'].str.contains('aapl', case=False) | 
                df_no_relevant_text['summary'].str.contains('apple', case=False) | df_no_relevant_text['summary'].str.contains('aapl', case=False)].copy()

In [ ]:
df_apple_in_ts.shape

(630, 16)

In [ ]:
# select those have apple or aapl in summary 

#dataset 2 for sentiment analysis
df_123 = df_apple_in_ts[df_apple_in_ts['summary'].str.contains('apple', case=False) | df_apple_in_ts['summary'].str.contains('aapl', case=False)].copy()


In [ ]:
df_123.shape

(527, 16)

In [ ]:
# finally, select those have apple or aapl in title but not in summary

# dataset 3 for sentiment analysis
df_456 = df_apple_in_ts[~(df_apple_in_ts['summary'].str.contains('apple', case=False) | df_apple_in_ts['summary'].str.contains('aapl', case=False))].copy()


In [ ]:
df_456.shape

(103, 16)

In [ ]:
# and also, the dataset that doesn't have relevant texts, and no apple or aapl in the title or summary

df_no_apple_aapl = df_no_relevant_text[~(df_no_relevant_text['title'].str.contains('apple', case=False) | df_no_relevant_text['title'].str.contains('aapl', case=False) | 
                df_no_relevant_text['summary'].str.contains('apple', case=False) | df_no_relevant_text['summary'].str.contains('aapl', case=False))].copy()



In [ ]:
df_no_apple_aapl.shape

(1587, 16)

In [ ]:
df_no_apple_aapl

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 109.9 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3) 
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone') 

nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer, device = 0, max_length=512, truncation=True)

In [ ]:
list1 = []

In [ ]:
for news in tqdm(df_apple_relevant_text['relevant_text']): 
   #try: 
     sentiment = nlp(news)[0]['label']  
     list1.append(sentiment)
   #except:
     #pass

  0%|          | 7/6046 [00:03<30:19,  3.32it/s]  /usr/local/lib/python3.9/dist-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 6046/6046 [03:16<00:00, 30.69it/s]


In [ ]:
list2 = []

In [ ]:
for news in tqdm(df_123['summary']): 
   #try: 
     sentiment = nlp(news)[0]['label']  
     list2.append(sentiment)
   #except:
     #pass

100%|██████████| 527/527 [00:05<00:00, 94.79it/s]


In [ ]:
list3 = []

In [ ]:
for news in tqdm(df_456['title']): 
   #try: 
     sentiment = nlp(news)[0]['label']  
     list3.append(sentiment)
   #except:
     #pass

100%|██████████| 103/103 [00:01<00:00, 82.56it/s]


In [ ]:
df_apple_relevant_text['finbert'] = list1

In [ ]:
df_123['finbert'] = list2

In [ ]:
df_456['finbert'] = list3

In [ ]:
df_456

In [ ]:
df_final = pd.concat([df_apple_relevant_text, df_123, df_456])

In [ ]:
df_final

,Unnamed: 0.1,Unnamed: 0,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment,relevant_text,finbert
0,0,0,"us stocks fall, oil tops $105 as ukraine crisi...",https://www.aljazeera.com/economy/2022/3/1/us-...,20220301T080000,['Rita Nazareth'],a surge in oil sent shivers through risky asse...,https://www.aljazeera.com/wp-content/uploads/2...,Al Jareeza,NaN,www.aljazeera.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",-0.277460,Somewhat-Bearish,"[{'ticker': 'AAPL', 'relevance_score': '0.0627...","['In the final moments of trading , the S & P ...",Negative
1,1,1,apple halts product sales in russia after ukra...,https://www.scmp.com/tech/big-tech/article/316...,20220302T020013,['Bloomberg'],apple inc halted sales of the iphone and its o...,https://img.i-scmp.com/cdn-cgi/image/fit=conta...,South China Morning Post,Companies,www.scmp.com,"[{'topic': 'Technology', 'relevance_score': '0...",-0.495467,Bearish,"[{'ticker': 'HPQ', 'relevance_score': '0.07817...",['Apple halts product sales in Russia after Uk...,Negative
2,2,2,russia says its economy is taking 'serious blo...,https://www.cnn.com/2022/03/02/business/russia...,20220302T080000,"['Mark Thompson and Vasco Cotovio', 'CNN Busin...",london/moscow ( cnn business ) the russian...,https://cdn.cnn.com/cnnnext/dam/assets/2203011...,CNN,Markets,www.cnn.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",-0.469431,Bearish,"[{'ticker': 'EADSF', 'relevance_score': '0.078...","['Apple , the world ’ s most valuable company ...",Negative
3,3,3,"rich russians turn to luxury jewellery, watche...",https://www.aljazeera.com/economy/2022/3/2/ric...,20220302T080000,"['Angelina Rascouet and Andy Hoffman', 'Flavia...",with sanctions on russia sending the ruble plu...,https://www.aljazeera.com/wp-content/uploads/2...,Al Jareeza,NaN,www.aljazeera.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",-0.118323,Neutral,"[{'ticker': 'AAPL', 'relevance_score': '0.0514...",['Even as consumer brands from Apple Inc. to N...,Neutral
5,5,5,why many businesses are getting tougher on rus...,https://www.cnn.com/2022/03/03/business/busine...,20220303T080000,['Chris Isidore'],why many businesses are getting tougher on rus...,https://media.cnn.com/api/v1/images/stellar/pr...,CNN,Markets,www.cnn.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",-0.321990,Somewhat-Bearish,"[{'ticker': 'AAPL', 'relevance_score': '0.1646...","[""Russian economy takes a hit as companies hal...",Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8005,8005,8564,this charging accessory is a must-have for app...,https://www.thestreet.com/deals/otterbox-apple...,20221220T210000,['Jacob Krol'],it's highly functional with a premium build an...,https://www.thestreet.com/.image/t_share/MTk0N...,The Street,NaN,www.thestreet.com,"[{'topic': 'Technology', 'relevance_score': '1...",0.227908,Somewhat-Bullish,"[{'ticker': 'AAPL', 'relevance_score': '0.8421...",[],Neutral
8071,8071,8637,best airpods deals: save on apple's earbuds at...,https://www.thestreet.com/deals/best-airpods-s...,20221222T215100,['Jacob Krol'],it's a good time to invest in a pair of airpods.,https://www.thestreet.com/.image/t_share/MTkyM...,The Street,NaN,www.thestreet.com,"[{'topic': 'Retail & Wholesale', 'relevance_sc...",0.379003,Bullish,"[{'ticker': 'AAPL', 'relevance_score': '0.2389...",[],Neutral
8119,8119,8692,apple iphone 14 pro supply bounces back after ...,https://www.investors.com/news/technology/ipho...,20221227T142600,"[""Investor's Business Daily"", 'PATRICK SEITZ']",iphone 14 pro supply bounces back after shorta...,https://www.investors.com/wp-content/uploads/2...,Investors Business Daily,NaN,www.investors.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.048269,Neutral,"[{'ticker': 'AAPL', 'relevance_score': '0.6873...",[],Positive
8157,8157,8736,"stock market live: stocks seek tesla, apple to..

In [ ]:
df_final['time_published'] = pd.to_datetime(df_final['time_published'], format='%Y%m%dT%H%M%S')

In [ ]:
df_final = df_final.sort_values(by='time_published')

In [ ]:
df_final = df_final.drop(df_final.columns[[0, 1]], axis=1)

In [ ]:
df_final

,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment,relevant_text,finbert
0,"us stocks fall, oil tops $105 as ukraine crisi...",https://www.aljazeera.com/economy/2022/3/1/us-...,2022-03-01 08:00:00,['Rita Nazareth'],a surge in oil sent shivers through risky asse...,https://www.aljazeera.com/wp-content/uploads/2...,Al Jareeza,NaN,www.aljazeera.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",-0.277460,Somewhat-Bearish,"[{'ticker': 'AAPL', 'relevance_score': '0.0627...","['In the final moments of trading , the S & P ...",Negative
1,apple halts product sales in russia after ukra...,https://www.scmp.com/tech/big-tech/article/316...,2022-03-02 02:00:13,['Bloomberg'],apple inc halted sales of the iphone and its o...,https://img.i-scmp.com/cdn-cgi/image/fit=conta...,South China Morning Post,Companies,www.scmp.com,"[{'topic': 'Technology', 'relevance_score': '0...",-0.495467,Bearish,"[{'ticker': 'HPQ', 'relevance_score': '0.07817...",['Apple halts product sales in Russia after Uk...,Negative
2,russia says its economy is taking 'serious blo...,https://www.cnn.com/2022/03/02/business/russia...,2022-03-02 08:00:00,"['Mark Thompson and Vasco Cotovio', 'CNN Busin...",london/moscow ( cnn business ) the russian...,https://cdn.cnn.com/cnnnext/dam/assets/2203011...,CNN,Markets,www.cnn.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",-0.469431,Bearish,"[{'ticker': 'EADSF', 'relevance_score': '0.078...","['Apple , the world ’ s most valuable company ...",Negative
3,"rich russians turn to luxury jewellery, watche...",https://www.aljazeera.com/economy/2022/3/2/ric...,2022-03-02 08:00:00,"['Angelina Rascouet and Andy Hoffman', 'Flavia...",with sanctions on russia sending the ruble plu...,https://www.aljazeera.com/wp-content/uploads/2...,Al Jareeza,NaN,www.aljazeera.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",-0.118323,Neutral,"[{'ticker': 'AAPL', 'relevance_score': '0.0514...",['Even as consumer brands from Apple Inc. to N...,Neutral
5,why many businesses are getting tougher on rus...,https://www.cnn.com/2022/03/03/business/busine...,2022-03-03 08:00:00,['Chris Isidore'],why many businesses are getting tougher on rus...,https://media.cnn.com/api/v1/images/stellar/pr...,CNN,Markets,www.cnn.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",-0.321990,Somewhat-Bearish,"[{'ticker': 'AAPL', 'relevance_score': '0.1646...","[""Russian economy takes a hit as companies hal...",Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8258,"even in an advertising slowdown, these 3 stock...",https://www.fool.com/investing/2022/12/31/even...,2022-12-31 12:00:00,['Adam Levy'],"ad sales growth will slow in 2023, but these s...",https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,NaN,www.fool.com,"[{'topic': 'Earnings', 'relevance_score': '0.9...",0.151938,Somewhat-Bullish,"[{'ticker': 'GOOG', 'relevance_score': '0.2433...",['able to track users across apps and the web ...,Neutral
8259,3 unstoppable growth stocks to buy after a sto...,https://www.fool.com/investing/2022/12/31/3-un...,2022-12-31 14:00:00,['Dani Cook'],these companies have grown by triple-digit per...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,NaN,www.fool.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.152163,Somewhat-Bullish,"[{'ticker': 'AMD', 'relevance_score': '0.22324...","[""Apple 's dependence on China for device prod...",Negative
8260,where will unity software stock be in 3 years?,https://www.fool.com/investing/2022/12/31/wher...,2022-12-31 15:45:00,['Leo Sun'],the game-engine developer still has a lot to p...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,NaN,www.fool.com,"[{'topic': 'IPO', 'relevance_score': '0.769861...",0.202065,Somewhat-Bullish,"[{'ticker': 'U', 'relevance_score': '0.095517'...",['Unity Ads also stopped working properly afte...,Neutral
8261,bulls in a b

In [ ]:
df_final.to_csv('drive/My Drive/thesis/news-aapl-sentiment.csv')

In [ ]:
from collections import Counter

# assume `my_list` is your list of elements
counts = Counter(list1)

# print the frequency counts
for elem, count in counts.items():
    print(elem, count)

Negative 467
Neutral 4489
Positive 1090


In [ ]:
# assume `my_list` is your list of elements
counts = Counter(list2)

# print the frequency counts
for elem, count in counts.items():
    print(elem, count)

Negative 110
Neutral 257
Positive 160


In [ ]:
# assume `my_list` is your list of elements
counts = Counter(list3)

# print the frequency counts
for elem, count in counts.items():
    print(elem, count)

Neutral 57
Negative 23
Positive 23


In [ ]:
df.overall_sentiment_label.value_counts()

Neutral             5115
Somewhat-Bullish    1834
Somewhat-Bearish     952
Bullish              210
Bearish              152
Name: overall_sentiment_label, dtype: int64